<div align="center">
    <H1>Aprendizado de Máquina - Trabalho Final</H1>
    <H3>Prof.º Daniel Roberto Cassar</H3> 
</div>

<br>

<div align="right">
    <H3>Guilda: Carcajus</H3>
    <H4>Aniel Souza Ribeiro Neto</H4>
    <H4>Caio Cogo Beriam</H4>
    <H4>Joaquim Junior Ferola Fonseca</H4>
</div>

<br> 

<div align="center">
    <img src="Imagens/carcajus_bg.png" width="500"/>
</div>

# Bibliotecas utilizadas

In [1]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

# Importando o dataset

In [2]:
df = pd.read_csv("nanotox_dataset.tsv", sep="\t")
df

,NPs,coresize,hydrosize,surfcharge,surfarea,Hsf,Ec,Ev,MeO,Cellline,...,ratio,e,esum,esumbyo,MW,NMetal,NOxygen,ox,viability,class
0,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,92.5258,nonToxic
1,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,96.1340,nonToxic
2,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,93.5567,nonToxic
3,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,97.6804,nonToxic
4,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,HCMEC,...,5.556,1.61,3.22,1.073,101.96,2,3,3,94.8454,nonToxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,127.4363,nonToxic
479,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,116.3751,nonToxic
480,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,40.8796,Toxic
481,ZnO,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,Caco2,...,2.703,1.65,1.65,1.650,81.38,1,1,2,86.8566,nonToxic


# Tratamento dos dados

In [3]:
df = df.dropna()
df = df.drop_duplicates()

In [4]:
colunas_categoricas = list(df.columns[df.dtypes == 'object'])
print(colunas_categoricas)

['NPs', 'Cellline', 'Celltype', 'class']


In [5]:
df = pd.get_dummies(df,
                    columns=colunas_categoricas,
                    dtype="int")

In [6]:
threshold = VarianceThreshold(0.1)
threshold.fit_transform(df.loc[:, df.dtypes != 'object'])
df

,coresize,hydrosize,surfcharge,surfarea,Hsf,Ec,Ev,MeO,Expotime,dosage,...,Cellline_HONDC,Cellline_L02,Cellline_MCF10A,Cellline_SHSY5Y,Cellline_SW480,Cellline_WI38,Celltype_Cancer,Celltype_Normal,class_Toxic,class_nonToxic
0,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.001,...,0,0,0,0,0,0,0,1,0,1
1,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.010,...,0,0,0,0,0,0,0,1,0,1
2,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.100,...,0,0,0,0,0,0,0,1,0,1
3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,1.000,...,0,0,0,0,0,0,0,1,0,1
4,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,5.000,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,1.000,...,0,0,0,0,0,0,1,0,0,1
479,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,10.000,...,0,0,0,0,0,0,1,0,0,1
480,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,100.000,...,0,0,0,0,0,0,1,0,1,0
481,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,24,0.100,...,0,0,0,0,0,0,1,0,0,1


In [7]:
class_dummies = df.filter(like='class_')
class_original = pd.from_dummies(class_dummies, sep='_')
df = pd.concat(
    [df.drop(class_dummies.columns, axis=1), class_original],
    axis=1
)
df

,coresize,hydrosize,surfcharge,surfarea,Hsf,Ec,Ev,MeO,Expotime,dosage,...,Cellline_HCMEC,Cellline_HONDC,Cellline_L02,Cellline_MCF10A,Cellline_SHSY5Y,Cellline_SW480,Cellline_WI38,Celltype_Cancer,Celltype_Normal,class
0,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.001,...,1,0,0,0,0,0,0,0,1,nonToxic
1,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.010,...,1,0,0,0,0,0,0,0,1,nonToxic
2,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,0.100,...,1,0,0,0,0,0,0,0,1,nonToxic
3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,1.000,...,1,0,0,0,0,0,0,0,1,nonToxic
4,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,24,5.000,...,1,0,0,0,0,0,0,0,1,nonToxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,1.000,...,0,0,0,0,0,0,0,1,0,nonToxic
479,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,10.000,...,0,0,0,0,0,0,0,1,0,nonToxic
480,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,48,100.000,...,0,0,0,0,0,0,0,1,0,Toxic
481,35.6,236.0,-41.6,27.9,-3.608,-3.89,-7.20,5.67,24,0.100,...,0,0,0,0,0,0,0,1,0,nonToxic


# Train-test split

In [8]:
semente = 39314

atributos = df.drop(columns=["class"])
target = df["class"]

x_treino, x_teste, y_treino, y_teste = train_test_split(
    atributos,
    target,
    test_size=0.1,
    random_state=semente,
    stratify=df["class"]
)